In [1]:
import json
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open("/home/user/pp_int/models/resnet/res_out_dir/results.json", "r") as f:
    res_obj = json.load(f)

In [3]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

in_gt_dir = "/mnt/nis_lab_research/data/coco_files/test/test1_neg_se_20"
with open(os.path.join(in_gt_dir, "result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [4]:
res_map = {}

for res in res_obj:
    fn = os.path.basename(res["file_name"])
    res_map[fn] = res

In [5]:
coco_gt_obj["categories"]

[{'id': 0, 'name': 'Accept Button'},
 {'id': 1, 'name': 'Advertisement'},
 {'id': 2, 'name': 'Alert Notification'},
 {'id': 3, 'name': 'Allow Button'},
 {'id': 4, 'name': 'Checkbox'},
 {'id': 5, 'name': 'Click Captcha'},
 {'id': 6, 'name': 'Close Button'},
 {'id': 7, 'name': 'Download Button'},
 {'id': 8, 'name': 'Email Input Box'},
 {'id': 9, 'name': 'Login Button'},
 {'id': 10, 'name': 'Logo'},
 {'id': 11, 'name': 'Name Input Box'},
 {'id': 12, 'name': 'Password Input Box'},
 {'id': 13, 'name': 'Phone Input Box'},
 {'id': 14, 'name': 'Play Button'},
 {'id': 15, 'name': 'Popup'},
 {'id': 16, 'name': 'Random'},
 {'id': 17, 'name': 'Submit Button'},
 {'id': 18, 'name': 'Toggle Button'},
 {'id': 19, 'name': 'Update Button'}]

In [6]:
# Group By Image

imgs_gt_pred_list = {}

tot_num_pred = 0
tot_num_gt = 0

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"].split("/")[-1]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, w, h, ann1["category_id"]])
            tot_num_gt += 1
    
    pred_list = []
    
    for ann2 in res_map[fn1]["annotations"]:
        bbox2 = ann2["bbox"]
        
        ann2_cat_id = ""
        for cat in coco_gt_obj["categories"]:
            if cat["name"] == ann2["category_id"]:
                ann2_cat_id = cat["id"]
                break
                
        bbox2.append(ann2_cat_id)
        pred_list.append(bbox2)
        tot_num_pred += 1
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [7]:
print(tot_num_pred)
print(tot_num_gt)

1827
273


In [8]:
keys = list(imgs_gt_pred_list.keys())


In [9]:
def calculate_iou(box1, box2):
    # Extract x, y, w, h from the input boxes
    
    x1 = box1[0]
    y1 = box1[1] 
    w1 = box1[2] 
    h1 = box1[3]
    
    x2 = box2[0]
    y2 = box2[1] 
    w2 = box2[2] 
    h2 = box2[3]
    
    # Determine the coordinates of the intersection rectangle
    x_intersect = max(x1, x2)
    y_intersect = max(y1, y2)
    w_intersect = min(x1 + w1, x2 + w2) - x_intersect
    h_intersect = min(y1 + h1, y2 + h2) - y_intersect
    
    # Ensure the intersection width and height are non-negative
    w_intersect = max(0, w_intersect)
    h_intersect = max(0, h_intersect)
    
    # Compute the area of intersection rectangle
    interArea = w_intersect * h_intersect
    
    # Compute the area of both the prediction and true bounding boxes
    box1Area = w1 * h1
    box2Area = w2 * h2
    
    # Compute the area of union
    unionArea = box1Area + box2Area - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea if unionArea > 0 else 0.0
    
    return iou


In [10]:
def calculate_int(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    return interArea

In [11]:
def draw_bounding_boxes(image_path, gt_bbox_list, pred_bbox_list, output_path, width=3):
    # Open the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Load a font
    font = ImageFont.load_default()

    # Draw bounding boxes and labels
    for bbox in gt_bbox_list:
        x, y, w, h, label = bbox
        draw.rectangle([x, y, x+w, y+h], outline="blue", width=width)
        draw.text((x, y), str(label), fill="blue", font=font)
        
    # Draw bounding boxes and labels
    for bbox in pred_bbox_list:
        x, y, w, h, label = bbox
        draw.rectangle([x, y, x+w, y+h], outline="red", width=width)
        draw.text((x, y), str(label), fill="red", font=font)

    # Save the new image
    image.save(output_path)

In [12]:
thold = .50

In [13]:
iou_scores = []

comp_base_dir = "./comp"
os.makedirs(comp_base_dir, exist_ok=True)

for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    
    draw_bounding_boxes(os.path.join(in_gt_dir, "images", key) , gts, preds, os.path.join(comp_base_dir, key)) 
    
    for i, pred in enumerate(preds):
        for j, gt in enumerate(gts):
            if int(pred[4]) == int(gt[4]):
                iou = calculate_iou(pred, gt)
                iou_scores.append([key, i, j, iou])
                
iou_scores_df = pd.DataFrame(iou_scores, columns=["fn", "pred_ind", "gt_ind", "iou"])

In [14]:
len(iou_scores_df[iou_scores_df["iou"] > 0])

187

In [15]:
iou_scores_df_g = iou_scores_df.groupby('fn')

In [16]:
iou_full_g_dfs = []
iou_corr_g_dfs = []
iou_dang_g_pred_dfs = []
iou_dang_g_gt_dfs = []
conf_mat = []

img_ctr = 0
pred_gtr_gt_list = []
gt_gtr_pred_list = []

for name, group in iou_scores_df_g:
    
    print(name)
    
    df_subset = group[["fn", "pred_ind", "gt_ind", "iou"]]
    df_subset = df_subset.sort_values(by="iou", ascending=False)
    df_subset = df_subset.reset_index(drop=True)
    
    iou_full_g_dfs.append(df_subset)
    tp = 0
    
    gt_list = list(df_subset["gt_ind"])
    pred_list = list(df_subset["pred_ind"])
        
    gt_set = list(set(gt_list))
    pred_set = list(set(pred_list))
    
    num_gt = len(gt_set)
    num_pred = len(pred_set)
    
    data_corr = []
    data_pred_dang = []
    data_gt_dang = []
    
    rem_flag = 0
    working_set = None
    
    # Number of predicted >= number of ground truths 
    if num_pred >= num_gt:
        pred_gtr_gt_list.append(img_ctr)
        working_set = pred_set
        rem_flag = 1
        
    # Number of ground truths > number of predicted    
    elif num_gt > num_pred:
        gt_gtr_pred_list.append(img_ctr)
        working_set = gt_set
        rem_flag = 2

    for i, row in df_subset.iterrows():
        
        if rem_flag == 1:
            if row["pred_ind"] in pred_set and row["iou"] > thold:
                data_corr.append(row)
                pred_set.remove(row["pred_ind"])
                tp += 1
            elif row["pred_ind"] in pred_set:
                data_pred_dang.append(row)
                pred_set.remove(row["pred_ind"])
            elif not pred_set:
                break
        elif rem_flag == 2:
            if row["gt_ind"] in gt_set and row["iou"] > thold:
                data_corr.append(row)
                gt_set.remove(row["gt_ind"])
                tp += 1
            elif row["gt_ind"] in gt_set:
                data_gt_dang.append(row)
                gt_set.remove(row["gt_ind"])
            elif not gt_set:
                break

    iou_corr_g_dfs.append(pd.DataFrame(data_corr))
    iou_dang_g_pred_dfs.append(pd.DataFrame(data_pred_dang))
    iou_dang_g_gt_dfs.append(pd.DataFrame(data_gt_dang))    
    
    fp = num_pred - tp
    fn = num_gt - tp
    
    conf_mat.append([tp, fp, fn])
    
    img_ctr += 1

0OIksf6yJXACtA9I-legalzoom_ss.png
1Rk264flsJusg9Jg-technicalgaurav_ss.png
1lO4s2Zt0zTFtW3U-amraandelma_ss.png
203FEtzEOm7dZ8Sz-dedicatedcore_ss.png
51Wgaq4GYYvAZH1H-ipcc_ss.png
5boADp7PkvOokhes-michaelschenkerhimself_ss.png
5dfgMzx6cj57loMN-em_ss.png
5nKtKAIuJS7qF9jc-ip2location_ss.png
5uBsPCCs5DBtpvq0-psu_ss.png
68O8MADx81sNrUrH-firsttactical_ss.png
75PoBRweHOvPeU1n-duosecurity_ss.png
8tavdem92XOxpUDE-whattheythink_ss.png
9iwsXaOHCQGoFO7J-medlineplus_ss.png
AhdC8513hQv3YrAT-biblegateway_ss.png
EuI0U0g2CmNB6scH-yabbycasino_ss.png
FzwcfJHvhY7NWzgs-makebelieveco_ss.png
GPNnwNGJ9DydyxpO-duke_ss.png
HwAFDBBbkgkpziTK-gameyum_ss.png
J0XY909gBZa6nutK-realcomm_ss.png
L9mvrtbV8iN3dpnz-clutchy_ss.png
L9oDbA9sni99lgdr-brightspace_ss.png
LgTCcgKP5TdxPSX4-gettyimages_ss.png
MtkJC8YU2AMBodgL-embark-studios_ss.png
NhQGolKrlBsuozDX-nasoya_ss.png
OREwbBWyTcmCdpD8-golden-road_ss.png
ORwrbnhoFWRdPieY-avg_ss.png
OUTKuwrroO01X3yP-corebook_ss.png
OvzuzAHWdefEjdZP-prednisolone_ss.png
PqYXAZ9HCb4czSDA-shopify

In [17]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    # Would be equivalent to correctly identifying areas where no object is present... so we set it to 0 because it is not relevant
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

In [18]:
print("tp_tot", tp_tot)
print("fp_tot", fp_tot)
print("tn_tot", tn_tot)
print("fn_tot", fn_tot)

tp_tot 130
fp_tot 976
tn_tot 0
fn_tot 82


In [19]:
print(len(pred_gtr_gt_list))

82


In [20]:
print(pred_gtr_gt_list)
print(gt_gtr_pred_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
[]


In [21]:
voi = 11

In [22]:
iou_full_g_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,8tavdem92XOxpUDE-whattheythink_ss.png,15,1,0.0


In [23]:
iou_corr_g_dfs[voi]

""


In [24]:
iou_dang_g_pred_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,8tavdem92XOxpUDE-whattheythink_ss.png,15,1,0.0


In [25]:
iou_dang_g_gt_dfs[voi]

""


In [26]:
accuracy = (tp_tot + tn_tot) / (tp_tot + fp_tot + tn_tot + fn_tot)
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print("accuracy: ", accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1_score: ", f1_score)

accuracy:  0.10942760942760943
precision:  0.11754068716094032
recall:  0.6132075471698113
f1_score:  0.19726858877086495


In [27]:
with open('./res_out_dir/metrics.txt', 'w+') as file:
    file.write(f"accuracy: {accuracy}\n")
    file.write(f"precision: {precision}\n")
    file.write(f"recall: {recall}\n")
    file.write(f"f1_score: {f1_score}\n")
